In [34]:
from sqlalchemy import create_engine
import pandas as pd

In [35]:
user = "schedulin"
passw = "MySQLIsFun"
host = "35.231.228.133"
database = "schedulin"

In [36]:
db = create_engine(
    'mysql+pymysql://{0}:{1}@{2}/{3}' \
        .format(user, passw, host, database), \
    connect_args = {'connect_timeout': 10})
conn = db.connect()

In [37]:
timetable_df = pd.read_sql_table(
    "timetable",
    con = conn)
resource_df = pd.read_sql_table(
    "resource",
    con = conn)
user_df = pd.read_sql_table(
    "user",
    con = conn)
reservation_df = pd.read_sql_table(
    "reservation",
    con = conn)

In [38]:
merge_1_df = resource_df.merge(timetable_df, left_on = "timetable_id", right_on = "id", suffixes=["_res", "_tim"])\
    .drop(axis = 1, columns = [
        "creation_date_res", "modification_date_res", "id_tim"])\
    .rename(columns={"id_res": "resource_id"})

In [39]:
merge_2_df = reservation_df.merge(user_df, left_on = "user_id", right_on = "id", suffixes=["_rva", "_use"])\
    .drop(axis = 1, columns = [
        "creation_date_rva", "modification_date_rva",
        "creation_date_use", "modification_date_use", "id_use"])\
    .rename(columns={"id_rva": "reservation_id"})

In [40]:
merge_3_df = merge_2_df.merge(merge_1_df, on = "resource_id",
                    suffixes = ["_rva", ""])\
    .drop(axis = 1, columns = [
        "deletion_date_rva", "deletion_date_res",
        "deletion_date_tim"])

In [41]:
import numpy as np

merge_3_df["is_legit"] = np.select([
    merge_3_df["date"] > \
        merge_3_df["deletion_date_use"]],
    [False],
    default = True
)

In [42]:
model_df = merge_3_df.groupby(["date"]).sum()[["is_legit"]].reset_index()
model_df

,date,is_legit
0,2012-11-04,11
1,2012-11-05,6
2,2012-11-06,7
3,2012-11-07,6
4,2012-11-08,11
...,...,...
3648,2022-10-31,49
3649,2022-11-01,56
3650,2022-11-02,51
3651,2022-11-03,56


In [43]:
model_df["day_of_week"] = model_df["date"].dt.day_name()
model_df["month"] = model_df["date"].dt.month_name()

In [44]:
def convert_to_season(column):
    if column in ["January","February","December"]:
        return "Winter"
    elif column in ["March","April","May"]:
        return "Spring"
    elif column in ["June","July","August"]:
        return "Summer"
    else:
        return "Autumn"

In [45]:
model_df["season"] = model_df["month"].apply(convert_to_season)

In [46]:
model_df["week_day"] = model_df["day_of_week"].apply(lambda x: "weekday" if x == "Monday" or x == "Tuesday"
                                                        or x == "Wednesday" or x == "Thursday" or x == "Friday"
                                                        else "weekend")

In [47]:
model_df.head()

,date,is_legit,day_of_week,month,season,week_day
0,2012-11-04,11,Sunday,November,Autumn,weekend
1,2012-11-05,6,Monday,November,Autumn,weekday
2,2012-11-06,7,Tuesday,November,Autumn,weekday
3,2012-11-07,6,Wednesday,November,Autumn,weekday
4,2012-11-08,11,Thursday,November,Autumn,weekday


In [48]:
import pandas as pd
df = pd.read_csv("temperature.csv")

In [49]:
df

,Unnamed: 0,time,apparent_temperature_max (°C),precipitation_sum (mm)
0,335,2012-12-01,3.3,0.0
1,336,2012-12-02,5.7,0.0
2,337,2012-12-03,7.3,0.0
3,338,2012-12-04,8.5,0.0
4,339,2012-12-05,6.0,0.0
...,...,...,...,...
3507,3842,2022-07-09,37.5,0.0
3508,3843,2022-07-10,38.4,0.0
3509,3844,2022-07-11,38.1,0.6
3510,3845,2022-07-12,37.7,0.0


In [50]:
df["time"] = pd.to_datetime(df["time"])

In [57]:
merge = model_df.merge(df, right_on="time", left_on="date")

In [53]:
merge.head()

,date,is_legit,day_of_week,month,season,week_day,Unnamed: 0,time,apparent_temperature_max (°C),precipitation_sum (mm)
0,2012-12-01,12,Saturday,December,Winter,weekend,335,2012-12-01,3.3,0.0
1,2012-12-02,10,Sunday,December,Winter,weekend,336,2012-12-02,5.7,0.0
2,2012-12-03,9,Monday,December,Winter,weekday,337,2012-12-03,7.3,0.0
3,2012-12-04,6,Tuesday,December,Winter,weekday,338,2012-12-04,8.5,0.0
4,2012-12-05,13,Wednesday,December,Winter,weekday,339,2012-12-05,6.0,0.0


In [58]:
merge = merge.drop(axis=1, columns=["Unnamed: 0","time"])

In [60]:
merge = merge.rename(columns={"is_legit":"reservations",
                    "apparent_temperature_max (°C)":"temp", "precipitation_sum (mm)":"prec"})

In [61]:
merge.head()

,date,reservations,day_of_week,month,season,week_day,temp,prec
0,2012-12-01,12,Saturday,December,Winter,weekend,3.3,0.0
1,2012-12-02,10,Sunday,December,Winter,weekend,5.7,0.0
2,2012-12-03,9,Monday,December,Winter,weekday,7.3,0.0
3,2012-12-04,6,Tuesday,December,Winter,weekday,8.5,0.0
4,2012-12-05,13,Wednesday,December,Winter,weekday,6.0,0.0


In [67]:
date_range = pd.date_range("2012-12-01", "2022-07-13")

In [71]:
#Comprobando que no hay fechas missing
print(len(date_range))
print(len(merge))

3512
3512
